# Scrapping de données météo

Dans ce notebook, on cherche à récupérer les données météo de plusieurs communes, afin de rendre pertinente une régression de la consommation d'énergie en fonction de l'indice de performance du bâtiment et de la météo.

Cette analyse s'inscrit dans une dimension bien plus locale que l'analyse précédente, qui portait sur une échelle départementale et non communale.

In [1]:
!pip install -q lxml

In [16]:
#!pip install -q lxml

import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm

from urllib import request

In [3]:
""" Création du dictionnaire Racine """

data_all = {
    i : {j : {} for j in range(1,13)} for i in range(2011,2019)
}

In [4]:
""" Scrapping du site infoclimat.fr et organisation des données sous forme de dictionnaire """
# Il faut optimiser la création en ne créant qu'une seule fois le Df lié aux villes

for i in tqdm(range(len(data_all))):
    for y in data_all :
        for m in data_all[y] :
            url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
            raw_text = request.urlopen(url_test).read()
            page = bs4.BeautifulSoup(raw_text) #,'lxml')
            tableau = page.find('table', {'id' : 'tableau-releves'})
            rows = tableau.find_all('tr')
    
            df = {
        'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
        'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
        'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
        'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
        'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
        'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
        'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
        "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
        'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                }
    
            data_all[y][m] = df

  0%|          | 0/8 [00:00<?, ?it/s]

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

In [23]:
#Création d'un Data Frame à partir de la colonne "Villes" du dictionnaire précédent
stations=pd.DataFrame(df['Villes'],columns=['Ville'])
#On ne garde que les stations dans les villes et pas celle de stations de "Météo à l'école" (MAE)
liste_communes=stations[~stations['Ville'].str[0].isin(['['])]
liste_communes

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
781,Xonrupt-Longemer (88)
782,Xonrupt-Longemer - Le Poli (88)
783,Xonrupt-Longemer - Refuge du Sotré (88)
784,Yssingeaux (43)


In [5]:
""" Conversion en DataFrame pour exporter """

df_all = pandas.DataFrame(data_all)

In [6]:
""" Exportation au format json """

df_all.to_json('data_base.json')

In [9]:
""" Normalisation des variables suivant la nomenclature de la source """

df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

In [10]:
class Meteo :
    def __init__(self):
        self.data_all = {
                i : {j : {} for j in range(1,13)} for i in range(2011,2019)
            }

    def scrap(self):
        for y in self.data_all :
            for m in self.data_all[y] :
                url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_test).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
        
                df = {
            'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
            'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
            'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
            'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
            'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
            'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
            'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
            "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
            'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                    }
        
                self.data_all[y][m] = df

    def export(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def rename(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)
    